#Temporal Difference Q-Learning
เรื่องนี้ค่อนข้างมีรายละเอียดทางคณิตศาสตร์พอสมควร สามารถเข้าไปฟังได้ที่ https://www.youtube.com/watch?v=vDDucTB6mig

ลง lib ที่จำเป็นต่าง ๆ ก่อน

In [ ]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.3 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
import numpy as np
import time
import random
import json
from tqdm import tqdm

ประกาศ environment หรือเกมจาก gymnasium โดยเกมที่เราเลือกคือ Frozen Lake ที่มีเป้าหมายว่าจะต้องไปเหยียบเส้นชัยโดยไม่ตกธารน้ำแข็ง

In [ ]:
#create the environment and customize the map
map = ["SFFF",
       "FHFH",
       "FFFH",
       "HFFG"]
map_np = np.array(list("".join(map)))
env = gym.make("FrozenLake-v1", desc=map, is_slippery=False)

กำหนดค่าตั้งต้นของ Q Table หรือก็คือโพยข้อสอบที่จะให้ AI เรียนรู้ จนสามารถเล่นเกมให้ผ่านได้

In [ ]:
#initialize q table
Q_table = np.random.uniform(-5, 5, (16,4))
Q_table[(map_np == 'H') | (map_np == 'G')] = 0
print('Value table:\n', Q_table)

#reward shaping
reward_mapping = {'S': -1, 'F': -1, 'H': -5, 'G': 10}
R_table = np.vectorize(reward_mapping.get, otypes=[float])(map_np)
print('Reward table:\n' ,R_table.reshape(4,4))

Value table:
 [[ 4.89249847  1.63801104  4.76583981 -3.53457715]
 [ 0.24780139  3.76887084 -4.49071354 -0.25802194]
 [ 4.50221641 -1.91132683  3.12977145  3.93414663]
 [-4.20230839 -2.58986839  4.77014024  3.64095288]
 [ 2.76477298  0.07151443 -4.8503271   4.67631422]
 [ 0.          0.          0.          0.        ]
 [-4.32852257  2.02717245 -4.02412365  1.61808459]
 [ 0.          0.          0.          0.        ]
 [-2.70535604  3.01955321  1.83616136 -3.29057235]
 [ 4.12536482  3.72276534 -2.24501529  1.0165261 ]
 [ 3.75443204 -1.18763345  3.54448611 -1.88797476]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-4.62137676  0.80315413  0.08267772  3.42096842]
 [ 1.60012748  3.20046163 -0.13002439 -0.63457967]
 [ 0.          0.          0.          0.        ]]
Reward table:
 [[-1. -1. -1. -1.]
 [-1. -5. -1. -5.]
 [-1. -1. -1. -5.]
 [-5. -1. -1. 10.]]


ส่วนต่อไปจะเป็นการเรียนรู้ ซึ่งจะเป็นการลองผิดลองถูก จน AI สามารถเดินไปหาเส้นชัยได้

In [ ]:
#set up hyperparameters
alpha = 0.9 #aka learning rate
gamma = 0.9 #discount factor
epoch = 1100
epsilon = 1
epsilon_decay_rate = 1e-3

#learning session
for i in tqdm(range(epoch), desc="Processing", unit="iteration"):
  state_0, info = env.reset()
  roll = np.random.default_rng()
  while True:
    action = env.action_space.sample() if roll.random() < epsilon else np.argmax(Q_table[state_0])

    state_1, _, terminated, truncated, info = env.step(action)
    reward = R_table[state_1]

    #update q table
    TD = reward + (gamma*np.max(Q_table[state_1])) - Q_table[state_0][action]
    Q_table[state_0][action] += alpha*TD

    state_0 = state_1

    if terminated or truncated:
      break

  epsilon = max(epsilon - epsilon_decay_rate, 0)
  if epsilon == 0:
    alpha = 0.0001

print('\n', Q_table)
with open('q_table.json', 'w') as json_file:
    json.dump(Q_table.tolist(), json_file)

Processing: 100%|██████████| 1100/1100 [00:00<00:00, 3231.68iteration/s]


 [[ 0.62882     1.8098      1.8098      0.62882   ]
 [ 0.62882    -5.          3.122       1.8098    ]
 [ 1.8098      4.58        1.8098      3.122     ]
 [ 3.122      -5.          1.80979722  1.8098    ]
 [ 1.8098      3.122      -5.          0.62882   ]
 [ 0.          0.          0.          0.        ]
 [-5.          6.2        -5.          3.122     ]
 [ 0.          0.          0.          0.        ]
 [ 3.122      -5.          4.58        1.8098    ]
 [ 3.122       6.2         6.2        -5.        ]
 [ 4.58        8.         -5.          4.58      ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-5.          6.2         8.          4.58      ]
 [ 6.2         8.         10.          6.2       ]
 [ 0.          0.          0.          0.        ]]


ผลลัพธ์ของการฝึก โดยมีเป้าหมายว่าจะต้องไปตกในช่องที่ 15 (เส้นชัย) ให้ได้

In [ ]:
state, info = env.reset()
print(state)

while True:
  action = np.argmax(Q_table[state])
  state, _, terminated, truncated, info = env.step(action)
  time.sleep(0.1)
  print(state)

  if terminated or truncated:
    break

env.close()

0
4
8
9
13
14
15


ซึ่งจะเห็นว่าท้ายที่สุด ตัว AI สามารถพาตัวมันเองจนรอดไปหาเส้นชัยหรือช่อง 15 ได้